In [39]:
import folktables
from folktables import ACSDataSource
import numpy as np

np.random.seed(0)
#(Age) must be greater than 16 and less than 90, and (Person weight) must be greater than or equal to 1
def employment_filter(data):
    """
    Filters for the employment prediction task
    """
    df = data
    df = df[df['AGEP'] > 16]
    df = df[df['AGEP'] < 90]
    df = df[df['PWGTP'] >= 1]
    return df
ACSEmployment = folktables.BasicProblem(
    features=[
       'AGEP', #age; for range of values of features please check Appendix B.4 of
           #Retiring Adult: New Datasets for Fair Machine Learning NeurIPS 2021 paper
       'SCHL', #educational attainment
       'MAR', #marital status
       'RELP', #relationship
       'DIS', #disability recode
       'ESP', #employment status of parents
       'CIT', #citizenship status
       'MIG', #mobility status (lived here 1 year ago)
       'MIL', #military service
       'ANC', #ancestry recode
       'NATIVITY', #nativity
       'DEAR', #hearing difficulty
       'DEYE', #vision difficulty
       'DREM', #cognitive difficulty
       'SEX', #sex
       'RAC1P', #recoded detailed race code
       'GCL', #grandparents living with grandchildren
    ],
    target='ESR', #employment status recode
    target_transform=lambda x: x == 1,
    group='DIS',
    preprocess=employment_filter,
    postprocess=lambda x: np.nan_to_num(x, -1),
)
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["FL"], download=True) #data for Florida state
features, label, group = ACSEmployment.df_to_numpy(acs_data)

In [40]:
from aif360.datasets import StandardDataset
import pandas as pd
data = pd.DataFrame(features, columns = ACSEmployment.features)
data['label'] = label
favorable_classes = [True]
protected_attribute_names = [ACSEmployment.group]
privileged_classes = np.array([[1]])
data_for_aif = StandardDataset(data, 'label', favorable_classes = favorable_classes,
                      protected_attribute_names = protected_attribute_names,
                      privileged_classes = privileged_classes)
privileged_groups = [{'DIS': 1}]
unprivileged_groups = [{'DIS': 2}]

/Users/Kotsi/Desktop/AI_Imperial/Ethics/Fairness-ML/my_env/lib/python3.11/site-packages/aif360/datasets/standard_dataset.py:143: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1.0' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[pos, label_name] = favorable_label


### Task 1(a) - Split data

In [41]:
data_size = data_for_aif.features.shape[0]
train, test = data_for_aif.split([0.7], shuffle=True)
splits = []
for _ in range(5):
    train, val = train.split([0.8], shuffle=True)
    splits.append((train, val))


### Task 1(b) - Model selection

In [54]:
from sklearn.preprocessing import StandardScaler  
from sklearn.linear_model import LogisticRegression


def find_most_accurate_logistic_reg_model(splits, parameters):
    final_scaler = None
    max_accuracy = -float('inf')
    best_model = None
    for parameter in parameters:
        accuracies = []
        for train, val in splits:
            scaler = StandardScaler()
            X_train = scaler.fit_transform(train.features)
            y_train = train.labels.ravel()

            X_val = scaler.transform(val.features)
            y_val = val.labels.ravel()
            learner = LogisticRegression(C=parameter, solver='liblinear', random_state=1)
            learner.fit(X_train, y_train)
            predictions = learner.predict(X_val)
            accuracy = sum((predictions==y_val) / len(y_val))
            accuracies.append(accuracy)
        mean_accuracy = sum(accuracies) / len(accuracies)
        if mean_accuracy > max_accuracy:
            max_accuracy = mean_accuracy
            best_model = learner
            final_scaler = scaler
    return best_model, final_scaler, max_accuracy



In [56]:
parameters = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
final_model, final_scaler, final_accuracy = find_most_accurate_logistic_reg_model(splits=splits, parameters=parameters)

In [58]:
print(f"Final Model: {final_model}\n")
print(f"Final Scaler: {final_scaler}\n")
print(f"Final Mean Accuracy on Val data : {final_accuracy}")

Final Model: LogisticRegression(C=0.1, random_state=1, solver='liblinear')

Final Scaler: StandardScaler()

Final Mean Accuracy on Val data : 0.751333899988068


### Test the final model on test data

In [63]:
from aif360.metrics import ClassificationMetric

X_test = final_scaler.transform(test.features)
y_test = test.labels.ravel()

test_predictions = final_model.predict(X_test)

test_pred = test.copy()
test_pred.labels = test_predictions

test_accuracy = sum((test_predictions == y_test) / len(y_test))

metric = ClassificationMetric(test, test_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
eq_opp_diff = metric.equal_opportunity_difference()

0.6253460276847063

### Final accuracy and fairness on test data

In [64]:
print(f"Final accuracy on test data: {test_accuracy}\n")
print(f"Fairness on test data using the equal opportunity difference metric: {eq_opp_diff}")

Final accuracy on test data: 0.7534268409311323

Fairness on test data using the equal opportunity difference metric: 0.6253460276847063


### Task(c) - Model selection based on fairness

In [80]:
def find_most_fair_logistic_reg_model(splits, parameters):
    final_scaler = None
    min_diff = float('inf')
    best_model = None
    for parameter in parameters:
        fairnesses = []
        for train, val in splits:
            scaler = StandardScaler()
            X_train = scaler.fit_transform(train.features)
            y_train = train.labels.ravel()

            X_val = scaler.transform(val.features)
            y_val = val.labels.ravel()
            learner = LogisticRegression(C=parameter, solver='liblinear', random_state=1)
            learner.fit(X_train, y_train)
            predictions = learner.predict(X_val)
            val_pred = val.copy()
            val_pred.labels = predictions
            metric = ClassificationMetric(val, val_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
            fairness = abs(metric.equal_opportunity_difference())
            fairnesses.append(fairness)
            
        mean_fairness = sum(fairnesses) / len(fairnesses)
        if mean_fairness < min_diff:
            min_diff = mean_fairness
            best_model = learner
            final_scaler = scaler
    return best_model, final_scaler, min_diff

In [81]:
final_model, final_scaler, final_fairness = find_most_fair_logistic_reg_model(splits = splits, parameters=parameters)

### Test the final model on test data

In [82]:
X_test = final_scaler.transform(test.features)
y_test = test.labels.ravel()

test_predictions = final_model.predict(X_test)

test_pred = test.copy()
test_pred.labels = test_predictions

test_accuracy = sum((test_predictions == y_test) / len(y_test))

metric = ClassificationMetric(test, test_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
eq_opp_diff = metric.equal_opportunity_difference()

### Final accuracy and fairness on test data

In [83]:
print(f"Final accuracy on test data: {test_accuracy}\n")
print(f"Fairness on test data using the equal opportunity difference metric: {eq_opp_diff}")

Final accuracy on test data: 0.7531678355119419

Fairness on test data using the equal opportunity difference metric: 0.6227677730313754
